In [ ]:
# YOLO PERSON Detector:

# === Import Libraries ===
import torch
import os
import time
import sys
import cv2
import gc
import numpy as np
np.float = float
import psutil
from tqdm.auto import tqdm 
from ultralytics import YOLO
from pathlib import Path

# === Use Torchreid's OSNet instead of FastReID ===
from torchreid.utils import FeatureExtractor

# === Ensure Torch Uses Metal (MPS) ===
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"✅ Using {device} for computation.")

# === Initialize YOLOv11 for Full-Body Detection ===
body_detector = YOLO("yolo11l.pt")  # Replace with smaller model if needed
device_str = "mps" if torch.backends.mps.is_available() else "cpu"
print("✅ YOLOv11 Person Detector Initialized.")

# === Initialize Torchreid OSNet Extractor ===
extractor = FeatureExtractor(
    model_name='osnet_x1_0',
    model_path="/Users/Christian/Torch ReID/osnet_x1_0_msmt17.pth",  
    device=device_str
)
print("✅ OSNet ReID Model Initialized on MPS.")

# === Tracker setup (if needed) ===
sys.path.append("/Users/Christian/Christian Home Drive/Christian/Projekte/CURRENTLY RUNNING PROJECTS/CV and NLP/Python_codes and apps/sort/BoT-SORT")

import importlib
import tracker.bot_sort_reid
importlib.reload(tracker.bot_sort_reid)

from tracker.bot_sort_reid import BoTSORT
import argparse

args = argparse.Namespace(
    track_high_thresh=0.7,
    track_low_thresh=0.04,
    new_track_thresh=0.24,
    track_buffer=7000,
    max_age=7000,
    n_init=4, 
    match_thresh=0.93,
    mot20=False,
    proximity_thresh=0.35,
    appearance_thresh=0.85,
    cmc_method="sparseOptFlow",
    name="BoT-SORT",  
    ablation=False,  
    with_reid=True,  
    lambda_=0.98,
    use_byte=False,
    device=device_str,
    imgsz=640
)

# === Set input and output folders ===
input_folder = "/Users/Christian/Downloads/Microadaptive Teaching Dritter Teil - LAs/Marlon/Erste Sitzung/YOLO" # Change PATH HERE!!!
output_folder = os.path.join(input_folder, "YOLO")
os.makedirs(output_folder, exist_ok=True)

video_files = [f for f in os.listdir(input_folder) if f.lower().endswith(".mp4")]

for video_file in video_files:
    input_video_path = os.path.join(input_folder, video_file)
    output_video_path = os.path.join(output_folder, f"{Path(video_file).stem}_tracked.mp4")
    csv_output_path = os.path.join(output_folder, f"{Path(video_file).stem}.csv")

    if os.path.exists(csv_output_path) and os.path.exists(output_video_path):
        print(f"⏭️ Skipping {video_file} (already processed)")
        continue

    print(f"\n▶️ Processing video: {video_file}")
    start_time = time.time()

    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        print(f"❌ Error: Failed to open video file {input_video_path}")
        sys.exit(1)

    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    progress_bar = tqdm(total=total_frames, desc="Processing Frames", unit="frame", leave=False)

    frame_num = 0
    columns = ['Frame', 'Person_ID', 'X1', 'Y1', 'X2', 'Y2']
    with open(csv_output_path, 'w') as f:
        f.write(','.join(columns) + '\n')

    tracker = BoTSORT(args, frame_rate=fps)

    with torch.no_grad():
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            frame_num += 1
            progress_bar.update(1)

            if frame_num % 100 == 0 and psutil.virtual_memory().percent > 80:
                gc.collect()
                if device_str == 'mps':
                    torch.mps.empty_cache()

            results = body_detector(frame, verbose=False, imgsz=640)
            tracked_bodies, body_crops = [], []

            for result in results:
                for box in result.boxes:
                    if int(box.cls[0].item()) != 0:
                        continue
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    conf = box.conf[0].item()
                    if conf > 0.15:
                        tracked_bodies.append([x1, y1, x2, y2, conf])
                        crop = frame[y1:y2, x1:x2]
                        if crop.shape[0] > 2 and crop.shape[1] > 2:
                            crop = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)
                            body_crops.append(crop)

            if len(body_crops) == 0:
                with open(csv_output_path, 'a') as f:
                    f.write(f"{frame_num},-1,-1,-1,-1,-1\n")  # placeholder for 'no detection'
                continue

            embeddings = extractor(body_crops)
            body_features = np.array([e.cpu().numpy() for e in embeddings])
            tracked_bodies_np = np.array(tracked_bodies, dtype=np.float64).reshape(-1, 5)

            try:
                tracked_results = tracker.update(tracked_bodies_np, frame, body_features)
            except Exception as e:
                print(f"⚠️ Tracking failed on frame {frame_num}: {e}")
                continue

            for track in tracked_results:
                x1, y1, w, h = map(int, track.tlwh)
                x2, y2 = x1 + w, y1 + h
                track_id = int(track.track_id)
                with open(csv_output_path, 'a') as f:
                    f.write(f"{frame_num},{track_id},{x1},{y1},{x2},{y2}\n")

    cap.release()
    progress_bar.close()
    gc.collect()
    print(f"✅ Done with {video_file}")


print("\n🎉 All sessions processed!")